Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события

- event - событие

    - app_install – установка приложения
    - app_start – открыть приложения
    - register – зарегистрироваться 
    - search – перейти на страницу поиска товаров (каталог)
    - open_item – открыть товар
    - choose_item – отправить товар в корзину
    - tap_basket – перейти в корзину
    - purchase – подтверждение покупки

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

    - yandex-direct – Яндекс директ
    - google_ads – реклама в Google
    - vk_ads – реклама в ВК
    - instagram_ads – реклама в instagram
    - facebook_ads – реклама в facebook
    - referal – акция «приведи друга»

Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  

- purchase_sum – стоимость покупки (при совершении события ‘purchase’)


Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 


### Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('KC_case_data.csv') 
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


### Tasks

1) Рассчитайте MAU февраля

In [3]:
mau_february = df.query('date>="2020-02-01"&date<="2020-02-29"').agg({'device_id':'nunique'})['device_id']
print(f"MAU за февраль 2020 года: {mau_february}")

MAU за февраль 2020 года: 75032


2) Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? 

Cчитаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [4]:
df.date=pd.to_datetime(df.date)
first_install=df.query('event=="app_install"').groupby('device_id').agg({'date':'min'}) #даты первого скачивания приложения пользователем
first_purchase=df.query('event=="purchase"').groupby('device_id').agg({'date':'min'}) #даты первой покупки пользователя

first_install.rename(columns={'date':'first_install_date'}, inplace=True)
first_purchase.rename(columns={'date':'first_purchase_date'}, inplace=True)

merged=df.merge(first_install, on='device_id', how='inner').merge(first_purchase, on='device_id', how='inner')

data=merged[['device_id', 'first_install_date', 'first_purchase_date']].drop_duplicates('device_id') 
data['days_since_installation']=(data.first_purchase_date-data.first_install_date).dt.days

purchases_in_7days = data.query('days_since_installation <=7 ')\
    .groupby('first_install_date').agg({'device_id': 'count'})\
    .sort_values('first_install_date')

all_installations=df.query('event=="app_install"').groupby('date').agg({'device_id': pd.Series.nunique})

conversion=round(purchases_in_7days*100/all_installations, 3)
conversion.rename(columns={'device_id':'devices_quantity'}, inplace=True)
max_conversion_date = conversion.devices_quantity.idxmax().strftime('%d-%m-%Y')
print(f'Для когорты, установившей приложение {max_conversion_date}, конверсия в покупку в течение 7 дней за эти 3 месяца является максимальной')

Для когорты, установившей приложение 01-01-2020, конверсия в покупку в течение 7 дней за эти 3 месяца является максимальной


3) С какого платного маркетингового канала пришло больше всего новых пользователей? («Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа»)

In [5]:
install_events = df.query('event=="app_install"')

paid_channels = install_events[install_events['utm_source']\
    .isin(['yandex-direct', 'google_ads', 'vk_ads', 'instagram_ads', 'facebook_ads', 'referal'])]

user_counts = paid_channels.groupby('utm_source')['device_id'].nunique()

top_channel = user_counts.idxmax()

channel_map = {
    'yandex-direct': 'Яндекс',
    'google_ads': 'Гугл',
    'vk_ads': 'ВК',
    'instagram_ads': 'Инстаграм',
    'facebook_ads': 'Фейсбук',
    'referal': 'Реферальная программа'
}

print(f'Среди новых пользователей больше всего пользователей пришло из "{channel_map[top_channel]}". Конкретно - {user_counts.max()}')

Среди новых пользователей больше всего пользователей пришло из "Яндекс". Конкретно - 29368


4) На каком этапе воронки отваливается бОльшая часть клиентов? Отличаются ли сценарии для зарегистрированных и для незарегистрированных пользователей? На каком шаге отваливается больше всего зарегистрированных пользователей?

- Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

In [81]:
df['date_of_registration']=df.query('event=="register"').date
registered=df[['device_id', 'date_of_registration']].set_index('device_id').dropna()
df=df.drop('date_of_registration', axis=1)
registered=df.merge(registered, on='device_id', how='inner')
registered

df_reg_completed=registered.query('date_of_registration<date') #записи по пользователям, которые на момент совершения действия уже были зарегистрированы

custom_order=['app_start','search', 'choose_item', 'tap_basket', 'purchase']

df_reg_completed['event']=pd.Categorical(df_reg_completed['event'], categories=custom_order, ordered=True)
df_reg_completed=df_reg_completed.sort_values('event')

df_reg_completed.groupby('event').agg({'device_id':'nunique'}).rename(columns={'device_id':'unique_users'})

/var/folders/yw/j_hz9v6107z3z2yhw4flcx6c0000gn/T/ipykernel_4932/4275378593.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg_completed['event']=pd.Categorical(df_reg_completed['event'], categories=custom_order, ordered=True)


,unique_users
event,
app_start,40991
search,40482
choose_item,37926
tap_basket,34517
purchase,24880


In [82]:
#сколько в абсолютном отношении "отсеивается" пользователей между шагами

df_reg_completed.groupby('event').agg({'device_id':'nunique'}).sort_values('device_id', ascending=False)\
                                .diff().rename(columns={'device_id':'absolute_change_in_unique_users'})

,absolute_change_in_unique_users
event,
app_start,NaN
search,-509.0
choose_item,-2556.0
tap_basket,-3409.0
purchase,-9637.0


In [83]:
#относительная конверсия 

res = 100+round(df_reg_completed.groupby('event').agg({'device_id':'nunique'}).sort_values('device_id', ascending=False)\
                                .pct_change()*100,3).rename(columns={'device_id':'conversion'})
res=res.reset_index()

res

,event,conversion
0,app_start,NaN
1,search,98.758
2,choose_item,93.686
3,tap_basket,91.011
4,purchase,72.080


#### 72% пользователей, когда-либо побывавших на сайте, купили товар от количества уникальных пользователей когда-либо перешедших в корзину => минимальной конверсией обладает действие "подтверждение покупки" относительно действия "переход в корзину"

5) Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [113]:
starts_by_channels=df.query('event=="app_start"').groupby('device_id').agg({'date':'min'}).reset_index()\
                     .merge(df, on=['device_id', 'date'], how='inner')\
                     .groupby('utm_source')\
                     .agg({'device_id':'nunique'})

first_purchase_by_channels=df.query('event=="purchase"').groupby('device_id').agg({'date':'min'}).reset_index()\
                     .merge(df, on=['device_id', 'date'], how='inner')\
                     .groupby('utm_source')\
                     .agg({'device_id':'nunique'})

first_purchase_conv=round(first_purchase_by_channels*100/starts_by_channels, 3).rename(columns={'device_id':'conversion'})

first_purchase_conv

min_channel = first_purchase_conv.idxmin()['conversion']
min_conv=first_purchase_conv['conversion'].min()

print(f'Наименьшая конверсия в первую покупку за эти 3 месяца наблюдается у такого канала как "{channel_map[min_channel]}".\nОна составляет {min_conv} %')

Наименьшая конверсия в первую покупку за эти 3 месяца наблюдается у такого канала как "Яндекс".
Она составляет 31.753 %


6) Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [115]:
med_check_df=df.merge(first_purchase, on='device_id', how='inner')
med_check_df=med_check_df.query('event=="purchase" and date==first_purchase_date')
med_check_df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,first_purchase_date
13,2020-01-07,purchase,269.0,android,669460,female,Moscow,-,2020-01-07
50,2020-01-01,purchase,329.5,ios,833621,male,Moscow,vk_ads,2020-01-01
167,2020-01-01,purchase,217.5,android,1579237,male,Saint-Petersburg,referal,2020-01-01
188,2020-01-24,purchase,198.5,android,5148476,male,Saint-Petersburg,google_ads,2020-01-24
281,2020-01-23,purchase,404.5,android,8383386,female,Saint-Petersburg,yandex-direct,2020-01-23
...,...,...,...,...,...,...,...,...,...
2122475,2020-03-31,purchase,567.5,android,21224405,female,Saint-Petersburg,yandex-direct,2020-03-31
2122482,2020-03-31,purchase,6963.0,android,11153353,female,Saint-Petersburg,-,2020-03-31
2122489,2020-03-31,purchase,165.5,android,14204302,male,Moscow,vk_ads,2020-03-31
2122496,2020-03-31,purchase,7542.0,android,18219505,female,Saint-Petersburg,-,2020-03-31


In [118]:
med_check_df.drop_duplicates()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,first_purchase_date
13,2020-01-07,purchase,269.0,android,669460,female,Moscow,-,2020-01-07
50,2020-01-01,purchase,329.5,ios,833621,male,Moscow,vk_ads,2020-01-01
167,2020-01-01,purchase,217.5,android,1579237,male,Saint-Petersburg,referal,2020-01-01
188,2020-01-24,purchase,198.5,android,5148476,male,Saint-Petersburg,google_ads,2020-01-24
281,2020-01-23,purchase,404.5,android,8383386,female,Saint-Petersburg,yandex-direct,2020-01-23
...,...,...,...,...,...,...,...,...,...
2122475,2020-03-31,purchase,567.5,android,21224405,female,Saint-Petersburg,yandex-direct,2020-03-31
2122482,2020-03-31,purchase,6963.0,android,11153353,female,Saint-Petersburg,-,2020-03-31
2122489,2020-03-31,purchase,165.5,android,14204302,male,Moscow,vk_ads,2020-03-31
2122496,2020-03-31,purchase,7542.0,android,18219505,female,Saint-Petersburg,-,2020-03-31


In [49]:
med_check_df.groupby('utm_source').agg({'purchase_sum':'median'}).sort_values('purchase_sum', ascending=False)

,purchase_sum
utm_source,
-,398.5
referal,395.5
instagram_ads,393.5
vk_ads,393.0
yandex-direct,392.5
google_ads,390.5
facebook_ads,389.0


Наибольший медианный чек выявлен у такого канала как "Реферальная программа": 395,5 руб. 

7) Данные по затратам на рекламу:
<br><br>
Яндекс – 10 491 707 руб. 
<br><br>
Гугл – 10 534 878 руб.
<br><br>
Фейсбук – 8 590 498 руб.
<br><br>
Инстаграм – 8 561626 руб.
<br><br>
ВК – 9 553 531руб.
<br><br>
Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.
Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 


$$
\text{ROMI} = \left( \frac{\text{Доход от канала} - \text{Затраты на канал}}{\text{Затраты на канал}} \right) \times 100\%
$$

In [168]:
revenue_by_channel = df.query('event=="purchase"').groupby('utm_source')\
         .agg({'purchase_sum':'sum'}).query('utm_source!="-" and utm_source!="referal"').rename(columns={'purchase_sum':'revenue'})

costs = {
    'yandex-direct': 10491707,
    'google_ads': 10534878,
    'vk_ads': 9553531,
    'instagram_ads': 8561626,
    'facebook_ads': 8590498
}

costs_dict={'utm_source': ['facebook_ads', 'google_ads', 'instagram_ads', 'vk_ads', 'yandex-direct'], 'marketing_costs': [8590498, 10534878, 8561626, 9553531, 10491707]}
marketing_costs=pd.DataFrame(costs_dict)

romi_df = revenue_by_channel.merge(marketing_costs, on='utm_source', how='inner').set_index('utm_source')
romi_df['ROMI'] = round((romi_df['revenue'] - romi_df['marketing_costs'])*100/romi_df['marketing_costs'], 3)

romi_df_max = romi_df['ROMI'].idxmax()

print(f'Наибольший ROMI фиксируется у такого канала как "{channel_map[romi_df_max]}"')

Наибольший ROMI фиксируется у такого канала как "ВК"


In [166]:
romi_df.sort_values('ROMI', ascending=False)

,revenue,marketing_costs,ROMI
utm_source,,,
vk_ads,16389652.5,9553531,71.556
instagram_ads,14546969.0,8561626,69.909
facebook_ads,12249901.0,8590498,42.598
yandex-direct,13915368.0,10491707,32.632
google_ads,12868276.0,10534878,22.149
